### Mathilde MOTTAY
# Apprentissage Automatique Numérique
# Mini-projet : classifieur bayésien

L'objectif de ce TP est de programmer un **classifieur bayésien**. 

## Chargement du jeu de données

On charge le jeu de données Iris et on affiche la partie **data** (description des données en termes d'attributs) et la partie **target** (classe, cible, étiquette).

In [ ]:
from sklearn import datasets 
iris = datasets.load_iris()
print ("Data :\n", iris.data)
print ("\nTarget :\n", iris.target)

## Division de l'échantillon d'apprentissage 

Les données étant triées par classe, on les mélange.

In [ ]:
import numpy as np
import scipy.stats

Ciris = np.c_[iris.data.reshape(len(iris.data), -1), iris.target.reshape(len(iris.target), -1)]
np.random.seed(987654321)
np.random.shuffle(Ciris)
shuffledIrisData = Ciris[ :, :iris.data.size//len(iris.data)].reshape(iris.data.shape)
shuffledIrisTarget = Ciris[ :, iris.data.size//len(iris.data) :].reshape(iris.target.shape)
print (shuffledIrisData)

On divise les données en trois parties : 
    - Apprentissage : 100 premiers exemples
    - Développement : 30 exemples suivants 
    - Test : 20 derniers exemples 

In [ ]:
# Apprentissage 
dataApprentissage = shuffledIrisData[:100]
targetApprentissage = shuffledIrisTarget[:100]
nbDonneesApprentissage = 100

# Développement 
dataDeveloppement = shuffledIrisData[100:130]
targetDeveloppement = shuffledIrisTarget[100:130]
nbDonneesDeveloppement = 30 

# Test
dataTest = shuffledIrisData[130:150]
targetTest = shuffledIrisTarget[130:150]
nbDonneesTest = 20

## Phase d'apprentissage

On utilise les données d'apprentissage et leur classe afin de déterminer les paramètres de notre classifieur Bayésien. 

- **Estimation des probabilités à priori pour les setosa, versicolor et virginica**

In [4]:
SETOSA = 0
VERSICOLOR = 1
VIRGINICA = 2

# Initialisation des compteurs setosa, versicolor et virginica à 0 
cptSetosa = 0
cptVersicolor = 0
cptVirginica = 0

# On calcule le nombre de setosa, versicolor et virginica en parcourant le tableau targetApprentissage.
# 0 --> setosa 
# 1 --> versicolor
# 2 --> virginica 
for i in targetApprentissage: 
    if i == SETOSA: 
        cptSetosa += 1
    elif i == VERSICOLOR: 
        cptVersicolor += 1
    elif i == VIRGINICA:
        cptVirginica += 1

print ("Il y a ", cptSetosa, "setosa, ", cptVersicolor, " versicolor et ", cptVirginica, " virginica.")

# Calcul des probabilités à priori pour i = {0, 1, 2} 
probaAPrioriSetosa = cptSetosa/nbDonneesApprentissage
probaAPrioriVersicolor = cptVersicolor/nbDonneesApprentissage
probaAPrioriVirginica = cptVirginica/nbDonneesApprentissage
print ("\nProbabilité à priori Setosa = ", probaAPrioriSetosa)
print ("Probabilité à priori Versicolor = ", probaAPrioriVersicolor)
print ("Probabilité à priori Virginica = ", probaAPrioriVirginica)

Il y a  32 setosa,  37  versicolor et  31  virginica.

Probabilité à priori Setosa =  0.32
Probabilité à priori Versicolor =  0.37
Probabilité à priori Virginica =  0.31


- **Estimation des vraisemblances**

Pour chaque classe, on obtient sa moyenne, sa variance et son écart-type.

In [5]:
COL_SEPAL_LENGTH = 0
COL_SEPAL_WIDTH = 1
COL_PETAL_LENGTH = 2
COL_PETAL_WIDTH = 3

# On commence par récupérer, dans 3 tableaux (un pour chaque classe), les caractéristiques des iris : 
# - la longueur des sépales (Sepal Length) --> indice 0
# - la largeur des sépales (Sepal Width) --> indice 1
# - la longueur des pétales (Petal Length) --> indice 2
# - la largeur des pétales (Petal Width) --> indice 3
setosaFeatures = [] 
versicolorFeatures = []
virginicaFeatures = []

for i in range(nbDonneesApprentissage):    
    if targetApprentissage[i] == SETOSA: 
        setosaFeatures.append([dataApprentissage[i][COL_SEPAL_LENGTH], dataApprentissage[i][COL_SEPAL_WIDTH], dataApprentissage[i][COL_PETAL_LENGTH], dataApprentissage[i][COL_PETAL_WIDTH]])
    elif targetApprentissage[i] == VERSICOLOR:
        versicolorFeatures.append([dataApprentissage[i][COL_SEPAL_LENGTH], dataApprentissage[i][COL_SEPAL_WIDTH], dataApprentissage[i][COL_PETAL_LENGTH], dataApprentissage[i][COL_PETAL_WIDTH]])
    elif targetApprentissage[i] == VIRGINICA:
        virginicaFeatures.append([dataApprentissage[i][COL_SEPAL_LENGTH], dataApprentissage[i][COL_SEPAL_WIDTH], dataApprentissage[i][COL_PETAL_LENGTH], dataApprentissage[i][COL_PETAL_WIDTH]])
        
print ("Caractéristiques : [sepal Length, sepal Width, petal Length, petal Width]")

# Calcul des moyennes (par classe) 
moySetosa = np.mean(setosaFeatures,0)
moyVersicolor = np.mean(versicolorFeatures,0)
moyVirginica = np.mean(virginicaFeatures,0)
print ("\nMoyenne Setosa  : " , moySetosa)
print ("Moyenne Versicolor : " , moyVersicolor)
print ("Moyenne Virginica : " , moyVirginica)

# Calcul des variances (par classe) 
print ("\nVariance Setosa : ", np.var(setosaFeatures,0, ddof = 1)) # ddof = 1 pour diviser par N-1
print ("Variance Versicolor ", np.var(versicolorFeatures,0, ddof = 1))
print ("Variance Virginica ", np.var(virginicaFeatures,0, ddof = 1))

# La covariance n'est pas utile car nous ne sommes pas dans un modèle bayésien naïf. 
# Par conséquent, la variance et la moyenne suffisent. 

# Calcul des écarts-types (par classe) 
stdSetosa = np.std(setosaFeatures,0, ddof = 1)
stdVersicolor = np.std(versicolorFeatures,0, ddof = 1)
stdVirginica = np.std(virginicaFeatures,0, ddof = 1)
print ("\nEcart-type Setosa : ", stdSetosa)
print ("Ecart-type Versicolor : ", stdVersicolor)
print ("Ecart-type Virginica : ", stdVirginica)

Caractéristiques : [sepal Length, sepal Width, petal Length, petal Width]

Moyenne Setosa  :  [4.971875 3.40625  1.459375 0.23125 ]
Moyenne Versicolor :  [5.82972973 2.75405405 4.16756757 1.28918919]
Moyenne Virginica :  [6.54516129 3.01612903 5.51290323 2.02903226]

Variance Setosa :  [0.10595766 0.12189516 0.03861895 0.00931452]
Variance Versicolor  [0.24436937 0.10144144 0.20336336 0.03376877]
Variance Virginica  [0.28589247 0.09073118 0.25916129 0.0727957 ]

Ecart-type Setosa :  [0.32551138 0.34913488 0.19651705 0.09651174]
Ecart-type Versicolor :  [0.4943373  0.31849873 0.45095827 0.18376281]
Ecart-type Virginica :  [0.53468914 0.30121617 0.50907886 0.26980678]


On suppose que les vraisemblances suivent une **loi Gaussienne**. On les stocke dans une matrice pour les utiliser facilement dans la phase de classification. 

In [6]:
matriceGaussiennes = np.zeros((3,4))

# Calcule les gaussiennes en utilisant les moyennes et écarts-types et les stocke dans la matrice 
def calculGaussiennes(sepalLength, sepalWidth, petalLength, petalWidth) :
    matriceGaussiennes[SETOSA][COL_SEPAL_LENGTH] = scipy.stats.norm.pdf(sepalLength,moySetosa[COL_SEPAL_LENGTH],stdSetosa[COL_SEPAL_LENGTH]) # Setosa - Sepal Length
    matriceGaussiennes[SETOSA][COL_SEPAL_WIDTH] = scipy.stats.norm.pdf(sepalWidth,moySetosa[COL_SEPAL_WIDTH],stdSetosa[COL_SEPAL_WIDTH]) # Setosa - Sepal Width 
    matriceGaussiennes[SETOSA][COL_PETAL_LENGTH] = scipy.stats.norm.pdf(petalLength,moySetosa[COL_PETAL_LENGTH],stdSetosa[COL_PETAL_LENGTH]) # Setosa - Petal Length
    matriceGaussiennes[SETOSA][COL_PETAL_WIDTH] = scipy.stats.norm.pdf(petalWidth,moySetosa[COL_PETAL_WIDTH],stdSetosa[COL_PETAL_WIDTH]) # Setosa - Petal Width 
    matriceGaussiennes[VERSICOLOR][COL_SEPAL_LENGTH] = scipy.stats.norm.pdf(sepalLength,moyVersicolor[COL_SEPAL_LENGTH],stdVersicolor[COL_SEPAL_LENGTH]) # Versicolor - Sepal Length
    matriceGaussiennes[VERSICOLOR][COL_SEPAL_WIDTH] = scipy.stats.norm.pdf(sepalWidth,moyVersicolor[COL_SEPAL_WIDTH],stdVersicolor[COL_SEPAL_WIDTH]) # Versicolor - Sepal Width 
    matriceGaussiennes[VERSICOLOR][COL_PETAL_LENGTH] = scipy.stats.norm.pdf(petalLength,moyVersicolor[COL_PETAL_LENGTH],stdVersicolor[COL_PETAL_LENGTH]) # Versicolor - Petal Length 
    matriceGaussiennes[VERSICOLOR][COL_PETAL_WIDTH] = scipy.stats.norm.pdf(petalWidth,moyVersicolor[COL_PETAL_WIDTH],stdVersicolor[COL_PETAL_WIDTH]) # Versicolor - Petal Width
    matriceGaussiennes[VIRGINICA][COL_SEPAL_LENGTH] = scipy.stats.norm.pdf(sepalLength,moyVirginica[COL_SEPAL_LENGTH],stdVirginica[COL_SEPAL_LENGTH]) # Virginica - Sepal Length
    matriceGaussiennes[VIRGINICA][COL_SEPAL_WIDTH] = scipy.stats.norm.pdf(sepalWidth,moyVirginica[COL_SEPAL_WIDTH],stdVirginica[COL_SEPAL_WIDTH]) # Virginica - Sepal Width 
    matriceGaussiennes[VIRGINICA][COL_PETAL_LENGTH] = scipy.stats.norm.pdf(petalLength,moyVirginica[COL_PETAL_LENGTH],stdVirginica[COL_PETAL_LENGTH]) # Virginica - Petal Length
    matriceGaussiennes[VIRGINICA][COL_PETAL_WIDTH] = scipy.stats.norm.pdf(petalWidth,moyVirginica[COL_PETAL_WIDTH],stdVirginica[COL_PETAL_WIDTH]) # Virginica - Petal Width

## Phase de développement

Pendant la phase de développement, on utilise des **modèles** (différentes variantes) pour classer les données. Dans chaque modèle on calcule les **probabilités à posteriori par classe** avec les paramètres déterminés lors de la phase d’apprentissage. On calcule ensuite le **taux d'erreurs** de chaque modèle puisqu'on dispose des classes correctes, pour déterminer le meilleur à utiliser pendant la phase d'évaluation. 

### 4 modèles à une caractéristique
- Modèle 1 : Longueur des sépales (Sepal Length)
- Modèle 2 : Largeur des sépales (Sepal Width)
- Modèle 3 : Longueur des pétales (Petal Length)
- Modèle 4 : Largeur des pétales (Petal Width)

In [7]:
probaAPosterioriParClasseModele1 = []
probaAPosterioriParClasseModele2 = []
probaAPosterioriParClasseModele3 = []
probaAPosterioriParClasseModele4 = []

# Calcule les probabilités à posteriori par classe pour un modèle à une caractéristique 
def modeleUneCaracteristique(c): 
    probaAPosterioriSetosa = matriceGaussiennes[SETOSA][c] * probaAPrioriSetosa
    probaAPosterioriVersicolor = matriceGaussiennes[VERSICOLOR][c] * probaAPrioriVersicolor
    probaAPosterioriVirginica = matriceGaussiennes[VIRGINICA][c] * probaAPrioriVirginica 
    
    # Modèle 1
    if c == COL_SEPAL_LENGTH: 
        probaAPosterioriParClasseModele1.append([probaAPosterioriSetosa, probaAPosterioriVersicolor, probaAPosterioriVirginica])
    # Modèle 2
    elif c == COL_SEPAL_WIDTH: 
        probaAPosterioriParClasseModele2.append([probaAPosterioriSetosa, probaAPosterioriVersicolor, probaAPosterioriVirginica])
    # Modèle 3
    elif c == COL_PETAL_LENGTH:
        probaAPosterioriParClasseModele3.append([probaAPosterioriSetosa, probaAPosterioriVersicolor, probaAPosterioriVirginica])
    # Modèle 4
    elif c == COL_PETAL_WIDTH: 
        probaAPosterioriParClasseModele4.append([probaAPosterioriSetosa, probaAPosterioriVersicolor, probaAPosterioriVirginica])

### 6 modèles à deux caractéristiques
- Modèle 5 - Longueur des sépales (Sepal Length) et largeur des sépales (Sepal Width) 
- Modèle 6 - Longueur des sépales (Sepal Length) et longueur des pétales (Petal Length)
- Modèle 7 - Longueur des sépales (Sepal Length) et largueur des pétales (Petal Width)
- Modèle 8 - Largeur des sépales (Sepal Width) et longueur des pétales (Petal Length) 
- Modèle 9 - Largeur des sépales (Sepal Width) et largeur des pétales (Petal Width)
- Modèle 10 - Longueur des pétales (Petal Length) et largeur des pétales (Petal Width)

In [8]:
probaAPosterioriParClasseModele5 = []
probaAPosterioriParClasseModele6 = []
probaAPosterioriParClasseModele7 = []
probaAPosterioriParClasseModele8 = []
probaAPosterioriParClasseModele9 = []
probaAPosterioriParClasseModele10 = []

# Calcule les probabilités à posteriori par classe pour un modèle à deux caractéristiques
def modeleDeuxCaracteristiques(c1, c2): 
    probaAPosterioriSetosa = matriceGaussiennes[SETOSA][c1] * matriceGaussiennes[SETOSA][c2] * probaAPrioriSetosa 
    probaAPosterioriVersicolor = matriceGaussiennes[VERSICOLOR][c1] * matriceGaussiennes[VERSICOLOR][c2] * probaAPrioriVersicolor 
    probaAPosterioriVirginica = matriceGaussiennes[VIRGINICA][c1] * matriceGaussiennes[VIRGINICA][c2] * probaAPrioriVirginica
    
    # Modèle 5 
    if ((c1 == COL_SEPAL_LENGTH) or (c2 == COL_SEPAL_LENGTH)) and ((c1 == COL_SEPAL_WIDTH) or (c2 == COL_SEPAL_WIDTH)): 
        probaAPosterioriParClasseModele5.append([probaAPosterioriSetosa, probaAPosterioriVersicolor, probaAPosterioriVirginica])
    # Modèle 6 
    elif ((c1 == COL_SEPAL_LENGTH) or (c2 == COL_SEPAL_LENGTH)) and ((c1 == COL_PETAL_LENGTH) or (c2 == COL_PETAL_LENGTH)):
        probaAPosterioriParClasseModele6.append([probaAPosterioriSetosa, probaAPosterioriVersicolor, probaAPosterioriVirginica])
    # Modèle 7
    elif ((c1 == COL_SEPAL_LENGTH) or (c2 == COL_SEPAL_LENGTH)) and ((c1 == COL_PETAL_WIDTH) or (c2 == COL_PETAL_WIDTH)):
        probaAPosterioriParClasseModele7.append([probaAPosterioriSetosa, probaAPosterioriVersicolor, probaAPosterioriVirginica])
    # Modèle 8
    elif ((c1 == COL_SEPAL_WIDTH) or (c2 == COL_SEPAL_WIDTH)) and ((c1 == COL_PETAL_LENGTH) or (c2 == COL_PETAL_LENGTH)):
        probaAPosterioriParClasseModele8.append([probaAPosterioriSetosa, probaAPosterioriVersicolor, probaAPosterioriVirginica])
    # Modèle 9
    elif ((c1 == COL_SEPAL_WIDTH) or (c2 == COL_SEPAL_WIDTH)) and ((c1 == COL_PETAL_WIDTH) or (c2 == COL_PETAL_WIDTH)): 
        probaAPosterioriParClasseModele9.append([probaAPosterioriSetosa, probaAPosterioriVersicolor, probaAPosterioriVirginica])
    # Modèle 10
    elif ((c1 == COL_PETAL_LENGTH) or (c2 == COL_PETAL_LENGTH)) and ((c1 == COL_PETAL_WIDTH) or (c2 == COL_PETAL_WIDTH)): 
        probaAPosterioriParClasseModele10.append([probaAPosterioriSetosa, probaAPosterioriVersicolor, probaAPosterioriVirginica])


### 4 modèles à trois caractéristiques
- Modèle 11 - Longueur des sépales (Sepal Length), largeur des sépales (Sepal Width) et longueur des pétales (Petal Length)
- Modèle 12 - Longueur des sépales (Sepal Length), largeur des sépales (Sepal Width) et largeur des pétales (Petal Width)
- Modèle 13 - Longueur des sépales (Sepal Length), longueur des pétales (Petal Length) et largeur des pétales (Petal Width)
- Modèle 14 - Largeur des sépales (Sepal Width), longueur des pétales (Petal Length) et largeur des pétales (Petal Width) 

In [9]:
probaAPosterioriParClasseModele11 = []
probaAPosterioriParClasseModele12 = []
probaAPosterioriParClasseModele13 = []
probaAPosterioriParClasseModele14 = []

# Calcule les probabilités à posteriori par classe pour un modèle à trois caractéristiques
def modeleTroisCaracteristiques(c1, c2, c3): 
    probaAPosterioriSetosa = matriceGaussiennes[SETOSA][c1] * matriceGaussiennes[SETOSA][c2] * matriceGaussiennes[SETOSA][c3] * probaAPrioriSetosa 
    probaAPosterioriVersicolor = matriceGaussiennes[VERSICOLOR][c1] * matriceGaussiennes[VERSICOLOR][c2] * matriceGaussiennes[VERSICOLOR][c3] * probaAPrioriVersicolor 
    probaAPosterioriVirginica = matriceGaussiennes[VIRGINICA][c1] * matriceGaussiennes[VIRGINICA][c2] * matriceGaussiennes[VIRGINICA][c3] * probaAPrioriVirginica

    # Modèle 11
    if(((c1 == COL_SEPAL_LENGTH) or (c2 == COL_SEPAL_LENGTH) or (c3 == COL_SEPAL_LENGTH)) and ((c1 == COL_SEPAL_WIDTH) or (c2 == COL_SEPAL_WIDTH) or (c3 == COL_SEPAL_WIDTH)) and ((c1 == COL_PETAL_LENGTH) or (c2 == COL_PETAL_LENGTH) or (c3 == COL_PETAL_LENGTH))): 
        probaAPosterioriParClasseModele11.append([probaAPosterioriSetosa, probaAPosterioriVersicolor, probaAPosterioriVirginica])
    # Modèle 12
    elif(((c1 == COL_SEPAL_LENGTH) or (c2 == COL_SEPAL_LENGTH) or (c3 == COL_SEPAL_LENGTH)) and ((c1 == COL_SEPAL_WIDTH) or (c2 == COL_SEPAL_WIDTH) or (c3 == COL_SEPAL_WIDTH)) and ((c1 == COL_PETAL_WIDTH) or (c2 == COL_PETAL_WIDTH) or (c3 == COL_PETAL_WIDTH))): 
        probaAPosterioriParClasseModele12.append([probaAPosterioriSetosa, probaAPosterioriVersicolor, probaAPosterioriVirginica])
    # Modèle 13
    elif(((c1 == COL_SEPAL_LENGTH) or (c2 == COL_SEPAL_LENGTH) or (c3 == COL_SEPAL_LENGTH)) and ((c1 == COL_PETAL_LENGTH) or (c2 == COL_PETAL_LENGTH) or (c3 == COL_PETAL_LENGTH)) and ((c1 == COL_PETAL_WIDTH) or (c2 == COL_PETAL_WIDTH) or (c3 == COL_PETAL_WIDTH))): 
        probaAPosterioriParClasseModele13.append([probaAPosterioriSetosa, probaAPosterioriVersicolor, probaAPosterioriVirginica])
    # Modèle 14
    elif(((c1 == COL_SEPAL_WIDTH) or (c2 == COL_SEPAL_WIDTH) or (c3 == COL_SEPAL_WIDTH)) and ((c1 == COL_PETAL_LENGTH) or (c2 == COL_PETAL_LENGTH) or (c3 == COL_PETAL_LENGTH)) and ((c1 == COL_PETAL_WIDTH) or (c2 == COL_PETAL_WIDTH) or (c3 == COL_PETAL_WIDTH))): 
        probaAPosterioriParClasseModele14.append([probaAPosterioriSetosa, probaAPosterioriVersicolor, probaAPosterioriVirginica])
       

### Un modèle à quatre caractéristiques
- Modèle 15 : Longueur des sépales (Sepal Length), largeur des sépales (Sepal Width), longueur des pétales (Petal Length) et largeur des pétales (Petal Width)

In [10]:
probaAPosterioriParClasseModele15 = []

# Calcule les probabilités à posteriori par classe pour un modèle à quatre caractéristiques
def modeleToutesLesCaracteristiques(): 
    probaAPosterioriSetosa = matriceGaussiennes[SETOSA][COL_SEPAL_LENGTH] * matriceGaussiennes[SETOSA][COL_SEPAL_WIDTH] * matriceGaussiennes[SETOSA][COL_PETAL_LENGTH] * matriceGaussiennes[SETOSA][COL_PETAL_WIDTH] * probaAPrioriSetosa 
    probaAPosterioriVersicolor = matriceGaussiennes[VERSICOLOR][COL_SEPAL_LENGTH] * matriceGaussiennes[VERSICOLOR][COL_SEPAL_WIDTH] * matriceGaussiennes[VERSICOLOR][COL_PETAL_LENGTH] * matriceGaussiennes[VERSICOLOR][COL_PETAL_WIDTH] * probaAPrioriVersicolor 
    probaAPosterioriVirginica = matriceGaussiennes[VIRGINICA][COL_SEPAL_LENGTH] * matriceGaussiennes[VIRGINICA][COL_SEPAL_WIDTH] * matriceGaussiennes[VIRGINICA][COL_PETAL_LENGTH] * matriceGaussiennes[VIRGINICA][COL_PETAL_WIDTH] * probaAPrioriVirginica
    probaAPosterioriParClasseModele15.append([probaAPosterioriSetosa, probaAPosterioriVersicolor, probaAPosterioriVirginica])

### Calcul du taux d'erreurs 

On code une fonction pour compter le taux d'erreurs puisqu'on connait la bonne classe pour chaque exemple du corpus. 
Elle nous permet ensuite de comparer les performances des différents modèles de notre classifieur.

In [11]:
# Calcule le taux d'erreurs 
def calculTauxErreurs (probaAPosterioriParClasse, nbValeurs, target): 
    erreur = 0
    for i in range(nbValeurs): 
        # La classe pour laquelle la probabilité a posteriori est MAXIMUM est la classe reconnue. 
        # Vérifie si le système a trouvé la bonne classe pour chaque exemple 
        if target[i] != np.argmax(probaAPosterioriParClasse[i]): 
            erreur += 1
    return ((erreur * 100) / nbValeurs)

### Utiliser les modèles pour classer les données

On traite séquentiellement tous les exemples du corpus de développement. 

In [12]:
# Affiche le taux d'erreurs de tous les modèles 
def afficheTauxErreursTousLesModeles(nbDonnees, target): 
    print ("Taux d'erreurs Modèle 1  : ", calculTauxErreurs(probaAPosterioriParClasseModele1, nbDonnees, target), "%")
    print ("Taux d'erreurs Modèle 2  : ", calculTauxErreurs(probaAPosterioriParClasseModele2, nbDonnees, target), "%")
    print ("Taux d'erreurs Modèle 3  : ", calculTauxErreurs(probaAPosterioriParClasseModele3, nbDonnees, target), "%")
    print ("Taux d'erreurs Modèle 4  : ", calculTauxErreurs(probaAPosterioriParClasseModele4, nbDonnees, target), "%")
    print ("Taux d'erreurs Modèle 5  : ", calculTauxErreurs(probaAPosterioriParClasseModele5, nbDonnees, target), "%")
    print ("Taux d'erreurs Modèle 6  : ", calculTauxErreurs(probaAPosterioriParClasseModele6, nbDonnees, target), "%")
    print ("Taux d'erreurs Modèle 7  : ", calculTauxErreurs(probaAPosterioriParClasseModele7, nbDonnees, target), "%")
    print ("Taux d'erreurs Modèle 8  : ", calculTauxErreurs(probaAPosterioriParClasseModele8, nbDonnees, target), "%")
    print ("Taux d'erreurs Modèle 9  : ", calculTauxErreurs(probaAPosterioriParClasseModele9, nbDonnees, target), "%")
    print ("Taux d'erreurs Modèle 10 : ", calculTauxErreurs(probaAPosterioriParClasseModele10, nbDonnees, target), "%")
    print ("Taux d'erreurs Modèle 11 : ", calculTauxErreurs(probaAPosterioriParClasseModele11, nbDonnees, target), "%")
    print ("Taux d'erreurs Modèle 12 : ", calculTauxErreurs(probaAPosterioriParClasseModele12, nbDonnees, target), "%")
    print ("Taux d'erreurs Modèle 13 : ", calculTauxErreurs(probaAPosterioriParClasseModele13, nbDonnees, target), "%")
    print ("Taux d'erreurs Modèle 14 : ", calculTauxErreurs(probaAPosterioriParClasseModele14, nbDonnees, target), "%")
    print ("Taux d'erreurs Modèle 15 : ", calculTauxErreurs(probaAPosterioriParClasseModele15, nbDonnees, target), "%")

# Vide les tableaux contenant les probabilités à posteriori (pour tous les modèles)
def videProbaAPosterioriTousModeles(): 
    probaAPosterioriParClasseModele1.clear()
    probaAPosterioriParClasseModele2.clear()
    probaAPosterioriParClasseModele3.clear()
    probaAPosterioriParClasseModele4.clear()
    probaAPosterioriParClasseModele5.clear()
    probaAPosterioriParClasseModele6.clear()
    probaAPosterioriParClasseModele7.clear()
    probaAPosterioriParClasseModele8.clear()
    probaAPosterioriParClasseModele9.clear()
    probaAPosterioriParClasseModele10.clear()
    probaAPosterioriParClasseModele11.clear()
    probaAPosterioriParClasseModele12.clear()
    probaAPosterioriParClasseModele13.clear()
    probaAPosterioriParClasseModele14.clear()
    probaAPosterioriParClasseModele15.clear()

# On traite séquentiellement tous les exemples du corpus de développement. 
def traitementCorpusDeveloppement(dataDeveloppement, nbDonneesDeveloppement): 
    videProbaAPosterioriTousModeles()

    for i in range(nbDonneesDeveloppement):
        calculGaussiennes(dataDeveloppement[i][COL_SEPAL_LENGTH], dataDeveloppement[i][COL_SEPAL_WIDTH], dataDeveloppement[i][COL_PETAL_LENGTH], dataDeveloppement[i][COL_PETAL_WIDTH])
        modeleUneCaracteristique(COL_SEPAL_LENGTH) # Modèle 1 
        modeleUneCaracteristique(COL_SEPAL_WIDTH) # Modèle 2 
        modeleUneCaracteristique(COL_PETAL_LENGTH) # Modèle 3
        modeleUneCaracteristique(COL_PETAL_WIDTH) # Modèle 4 
        modeleDeuxCaracteristiques(COL_SEPAL_LENGTH,COL_SEPAL_WIDTH) # Modèle 5 
        modeleDeuxCaracteristiques(COL_SEPAL_LENGTH,COL_PETAL_LENGTH) # Modèle 6 
        modeleDeuxCaracteristiques(COL_SEPAL_LENGTH,COL_PETAL_WIDTH) # Modèle 7 
        modeleDeuxCaracteristiques(COL_SEPAL_WIDTH,COL_PETAL_LENGTH) # Modèle 8 
        modeleDeuxCaracteristiques(COL_SEPAL_WIDTH,COL_PETAL_WIDTH) # Modèle 9 
        modeleDeuxCaracteristiques(COL_PETAL_LENGTH,COL_PETAL_WIDTH) # Modèle 10
        modeleTroisCaracteristiques(COL_SEPAL_LENGTH,COL_SEPAL_WIDTH,COL_PETAL_LENGTH) # Modèle 11 
        modeleTroisCaracteristiques(COL_SEPAL_LENGTH,COL_SEPAL_WIDTH,COL_PETAL_WIDTH) # Modèle 12 
        modeleTroisCaracteristiques(COL_SEPAL_LENGTH,COL_PETAL_LENGTH,COL_PETAL_WIDTH) # Modèle 13 
        modeleTroisCaracteristiques(COL_SEPAL_WIDTH,COL_PETAL_LENGTH,COL_PETAL_WIDTH) # Modèle 14 
        modeleToutesLesCaracteristiques() # Modèle 15

traitementCorpusDeveloppement(dataDeveloppement,nbDonneesDeveloppement)
afficheTauxErreursTousLesModeles(nbDonneesDeveloppement,targetDeveloppement)

Taux d'erreurs Modèle 1  :  26.666666666666668 %
Taux d'erreurs Modèle 2  :  46.666666666666664 %
Taux d'erreurs Modèle 3  :  6.666666666666667 %
Taux d'erreurs Modèle 4  :  10.0 %
Taux d'erreurs Modèle 5  :  20.0 %
Taux d'erreurs Modèle 6  :  13.333333333333334 %
Taux d'erreurs Modèle 7  :  10.0 %
Taux d'erreurs Modèle 8  :  10.0 %
Taux d'erreurs Modèle 9  :  10.0 %
Taux d'erreurs Modèle 10 :  13.333333333333334 %
Taux d'erreurs Modèle 11 :  16.666666666666668 %
Taux d'erreurs Modèle 12 :  13.333333333333334 %
Taux d'erreurs Modèle 13 :  10.0 %
Taux d'erreurs Modèle 14 :  13.333333333333334 %
Taux d'erreurs Modèle 15 :  13.333333333333334 %


On observe que le modèle avec le moins d'erreurs est le **modèle N°3**. 
Il utilise une seule caractéristique : la **longueur des pétales** (Petal Length).

On décide de garder ce modèle puisque c'est la variante dont le taux d'erreur est **minimal** (6.666666666666667 %). 

## Phase d'évaluation

On applique notre **meilleur modèle (modèle 3)** sur les données de Test. Pour rappel, il n'utilise qu'une seule caractéristique : la longueur des pétales. 

In [13]:
videProbaAPosterioriTousModeles()

for i in range(nbDonneesTest):
    calculGaussiennes(dataTest[i][COL_SEPAL_LENGTH], dataTest[i][COL_SEPAL_WIDTH], dataTest[i][COL_PETAL_LENGTH], dataTest[i][COL_PETAL_WIDTH])
    modeleUneCaracteristique(COL_PETAL_LENGTH) # Modèle 3 - Petal Length
        
print ("Taux d'erreurs à la phase d'évaluation : ", calculTauxErreurs(probaAPosterioriParClasseModele3, nbDonneesTest, targetTest), "%")

Taux d'erreurs à la phase d'évaluation :  15.0 %


Le taux d'erreurs à la phase d'évaluation est de **15%**.